In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from dataset import *
from model import GoogLeNet
import torchvision.transforms as transforms

In [2]:
data_transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.RandomCrop(size=(224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [3]:
train_data = HAM10000Dataset('HAM10000_metadata.csv', 'HAM10000_images_part_1', 'HAM10000_images_part_2', transform=data_transform)

In [4]:
train_dataloader = DataLoader(train_data, 64, True)

In [22]:
x, y = next(iter(train_dataloader))
x.shape, y.shape

(torch.Size([64, 3, 224, 224]), torch.Size([64, 7]))

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [7]:
model1 = GoogLeNet(in_channels=3, num_features=7).to(device)

In [8]:
model1.load_state_dict(torch.load(f='models/checkpoint.pth'))

<All keys matched successfully>

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)

In [9]:
model1.state_dict()

OrderedDict([('conv1.conv.weight',
              tensor([[[[ 1.6301e-02,  5.7611e-02, -7.0517e-03,  ..., -2.7404e-02,
                         -4.6535e-05,  1.2977e-02],
                        [-7.6741e-02, -2.4023e-02,  8.6575e-03,  ...,  9.7970e-03,
                         -5.9341e-02,  6.6101e-02],
                        [-5.3242e-02, -7.4214e-03, -1.3124e-02,  ..., -4.4727e-02,
                          1.5384e-02,  5.2653e-02],
                        ...,
                        [ 5.9101e-02, -8.8993e-02,  1.3082e-02,  ..., -4.0751e-02,
                          5.8000e-02, -2.2212e-02],
                        [ 8.3175e-02,  6.9550e-02,  1.5561e-02,  ..., -4.4917e-02,
                         -2.1623e-02,  3.2384e-02],
                        [-1.7448e-02,  1.8686e-02,  4.5451e-02,  ..., -5.3770e-02,
                         -4.8115e-02, -7.5641e-03]],
              
                       [[ 6.0722e-02, -9.8030e-02,  3.3495e-02,  ..., -1.7379e-02,
                         -9

In [11]:
model1(x.to(device)).shape

torch.Size([64, 7])

In [12]:
from tqdm.auto import tqdm

In [13]:
epochs = 50

for epoch in tqdm(range(epochs)):
    model1.train()
    train_loss = 0
    train_acc = 0
    correct = 0
    total = 0

    for batch_idx, (x, y) in enumerate(train_dataloader):
        x = x.to(device)
        y = y.to(device)

        y_logits = model1(x).squeeze()
        loss = loss_fn(y_logits, y)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        y_preds = torch.softmax(y_logits, dim=1).argmax(dim=1)
        correct += (y_preds == y.argmax(dim=1)).sum().item()
        total += y.shape[0]

    torch.save(obj=model1.state_dict(), f='models/checkpoint.pth')

    train_loss /= len(train_dataloader)
    train_acc = correct / total
    print(f'epoch: {epoch}')
    print(f'train_loss: {train_loss} | train_acc: {train_acc}')

  0%|          | 0/50 [00:00<?, ?it/s]

epoch: 0
train_loss: 0.2077559656018664 | train_acc: 0.9225162256615077
epoch: 1
train_loss: 0.1955939889381266 | train_acc: 0.9248127808287568
epoch: 2
train_loss: 0.18209126665239123 | train_acc: 0.9332001997004493
epoch: 3
train_loss: 0.18063799334559472 | train_acc: 0.9312031952071892
epoch: 4
train_loss: 0.15652351370852463 | train_acc: 0.9423864203694459
epoch: 5
train_loss: 0.17208374426907794 | train_acc: 0.9371942086869696
epoch: 6
train_loss: 0.1416640485500454 | train_acc: 0.9473789316025961
epoch: 7
train_loss: 0.15080252563830016 | train_acc: 0.9418871692461308
epoch: 8
train_loss: 0.13631506436237484 | train_acc: 0.9489765351972042
epoch: 9
train_loss: 0.12980461788547645 | train_acc: 0.9520718921617574
epoch: 10
train_loss: 0.12581247914653676 | train_acc: 0.9533699450823764
epoch: 11
train_loss: 0.1222710882663537 | train_acc: 0.9539690464303545
epoch: 12
train_loss: 0.10777972544881569 | train_acc: 0.962156764852721
epoch: 13
train_loss: 0.11971213116909668 | train_acc

In [23]:
y 

tensor([[0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0.,

In [24]:
y_preds = model1(x.to(device))
y_preds = torch.argmax(y_preds, dim=1)
y = torch.argmax(y, dim=1)

In [36]:
correct = 0
correct = (y.to(device) == y_preds).sum().item()
correct / y.shape[0]

1.0